## 1. Libraries

In [1]:
import pandas as pd
import gspread

## 2. Functions

In [2]:
def get_type(transaction):
    pay_method = transaction.split(" ")[0]

    if pay_method == "Pago":
        return_type = " ".join(transaction.split(" ")[0:2])
    elif pay_method == "Transaccion":
        return_type = " ".join(transaction.split(" ")[0:2])
    else:
        return_type = transaction.split(" ")[0]
    
    return return_type

In [3]:
def get_detail(transaction):
    payment_split = transaction.split(" ")
    pay_method = payment_split[0]
    
    return_detail = ""

    if pay_method == "Pago":
        return_detail = " ".join(transaction.split(" ")[3:]).split(",")[0]
    elif pay_method == "Compra":
        return_detail = " ".join(transaction.split(" ")[1:]).split(",")[0]
    elif pay_method == "Transferencia":
        if payment_split[1] == "A":
            return_detail = " ".join(payment_split[4:])
        elif payment_split[1] == "De":
            return_detail = " ".join(payment_split[2:])
    elif pay_method == "Bizum":
        return_detail = " ".join(payment_split[2:])
    elif pay_method == "Transaccion":
        return_detail = " ".join(transaction.split(" ")[3:]).split(",")[0]
    elif pay_method == "Recibo":
        return_detail = " ".join(payment_split[1:3])

    return return_detail

## 3. Creating the main table

In [4]:
df = pd.read_excel("raw_excels/2023_09.xls")
df.columns = df.iloc[6]
df = df.iloc[7:].reset_index(drop=True)

In [5]:
df["type_01"] = df["CONCEPTO"].apply(lambda x : x.split(" ")[0])
df["type_02"] = df["CONCEPTO"].apply(lambda x : x.split(" ")[1]+" "+x.split(" ")[2])

In [6]:
payments = df[['FECHA OPERACIÓN', 'FECHA VALOR', 'CONCEPTO', 'IMPORTE EUR', 'SALDO']]
payments["payment_type"] = payments["CONCEPTO"].apply(get_type)
payments["payment_detail"] = payments["CONCEPTO"].apply(get_detail)
payments["payment_concepto"] = payments["payment_detail"].apply(lambda x : x.split("Concepto")[1] if "Concepto" in x else "")
payments["payment_detail"] = payments["payment_detail"].apply(lambda x : x.split("Concepto")[0])

C:\Users\santi\AppData\Local\Temp\ipykernel_29412\2546723088.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  payments["payment_type"] = payments["CONCEPTO"].apply(get_type)


In [7]:
payments.loc[payments['payment_concepto'].str.contains("Nomina", case=False, na=False), 'payment_type'] = 'Nomina'

In [8]:
payments

6,FECHA OPERACIÓN,FECHA VALOR,CONCEPTO,IMPORTE EUR,SALDO,payment_type,payment_detail,payment_concepto
0,29/09/2023,29/09/2023,"Transferencia De Esml Sd Iberia Holding, Sa, C...",204.86,206.06,Nomina,"Esml Sd Iberia Holding, Sa,","Tr Nomina Esml Sd Iberia Holding, Sa."
1,28/09/2023,28/09/2023,"Compra Internet En Delikia Cyberpa, Gondomar, ...",-0.95,1.2,Compra,Internet En Delikia Cyberpa,
2,27/09/2023,22/09/2023,"Compra Leon -camp. 19:09:20, Llanera, Tarjeta ...",-14.3,2.15,Compra,Leon -camp. 19:09:20,
3,25/09/2023,23/09/2023,"Compra Artesania Picos De Eu, La Riera De C, T...",-4.3,16.45,Compra,Artesania Picos De Eu,
4,25/09/2023,22/09/2023,"Compra Paypal *uberpayment, 35314369001, Tarje...",-18.12,20.75,Compra,Paypal *uberpayment,
...,...,...,...,...,...,...,...,...
58,04/09/2023,02/09/2023,"Compra Zettle_*goiko, Madrid, Tarjeta 51638301...",-9.5,273.65,Compra,Zettle_*goiko,
59,04/09/2023,01/09/2023,"Transaccion Contactless En 1 2 3 Wok, Madrid, ...",-9.95,283.15,Transaccion Contactless,1 2 3 Wok,
60,04/09/2023,01/09/2023,"Transaccion Contactless En Bowling La Ermi, Ma...",-33,293.1,Transaccion Contactless,Bowling La Ermi,
61,01/09/2023,01/09/2023,"Transaccion Contactless En Sup.ex. Princip, Ma...",-2.85,326.1,Transaccion Contactless,Sup.ex. Princip,


In [9]:
transactions = payments.drop(["CONCEPTO","FECHA VALOR","SALDO"],axis=1)
#transactions.to_csv("clean_excels/transactions_2023_09.csv",index=False)

In [10]:
df.columns

Index(['FECHA OPERACIÓN', 'FECHA VALOR', 'CONCEPTO', 'IMPORTE EUR', 'SALDO',
       'type_01', 'type_02'],
      dtype='object', name=6)

In [17]:
transactions[:50]

6,FECHA OPERACIÓN,IMPORTE EUR,payment_type,payment_detail,payment_concepto
0,29/09/2023,204.86,Nomina,"Esml Sd Iberia Holding, Sa,","Tr Nomina Esml Sd Iberia Holding, Sa"
1,28/09/2023,-0.95,Compra,Internet En Delikia Cyberpa,
2,27/09/2023,-14.3,Compra,Leon -camp. 19:09:20,
3,25/09/2023,-4.3,Compra,Artesania Picos De Eu,
4,25/09/2023,-18.12,Compra,Paypal *uberpayment,
5,25/09/2023,-17.38,Compra,Paypal *uberpayment,
6,25/09/2023,-3.86,Impuesto:,,
7,25/09/2023,-5.41,Pago Movil,La Despensa T18,
8,25/09/2023,40,Ingreso,,
9,25/09/2023,-20,"Reintegro,",,


In [12]:


gc = gspread.service_account()

sh = gc.open("Personal Finances")

print(sh.sheet1.get('A1'))

[]


In [13]:
wks = sh.worksheet("Raw Data")
wks.insert_row([1,4,0], 10)

WorksheetNotFound: Raw Data